<h3>This ipynb book purpose is to convert existing model into tflite format to allow integration into Android app</h3>

In [ ]:
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb
from keras.models import load_model
import tensorflow as tf
import os

In [ ]:
# Loading model to convert
model = load_model("textReadModel(handwritten).keras")
model.summary()

In [ ]:
img_size = 32

# Creates model info.
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "OCR Model"
model_meta.description = ("OCR model to recognize handwritten characters")
model_meta.version = "v3.1, 01/12/2023"
model_meta.author = "M.Fokins & A.Bistrovs"

# Creating input info.
input_meta = _metadata_fb.TensorMetadataT()
input_meta.name = "image"
input_meta.description = (
    "Input image to be classified. The expected image is {0} x {1}"
    ", with 1 color channel (grayscale). Each value in the tensor is a single"
    " pixel value (uint8 between 0 to 255).".format(img_size, img_size))
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (
    _metadata_fb.ColorSpaceType.GRAYSCALE)
input_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.ImageProperties)
input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (
    _metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = [0.5]
input_normalization.options.std = [0.5]
input_meta.processUnits = [input_normalization]
input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats

# Creating output info.
output_meta = _metadata_fb.TensorMetadataT()
output_meta.name = "probability"
output_meta.description = "Probabilities of the 25 possible character respectively."
output_meta.content = _metadata_fb.ContentT()
output_meta.content.content_properties = _metadata_fb.FeaturePropertiesT()
output_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_stats = _metadata_fb.StatsT()
output_stats.max = [1.0]
output_stats.min = [0.0]
output_meta.stats = output_stats
label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename("label_classes.npy")
label_file.description = "List of the 25 characters."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_VALUE_LABELS
output_meta.associatedFiles = [label_file]

# Creating subgraph info.
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_meta]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

# Converting model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Saving model.
with open('textReadModel(handwritten).tflite', 'wb') as f:
  f.write(tflite_model)

# Populating model file with metadata.
populator = _metadata.MetadataPopulator.with_model_file('textReadModel(handwritten).tflite')
populator.load_metadata_buffer(metadata_buf)
populator.load_associated_files(["label_classes.npy"])
populator.populate()

In [ ]:
# Visualising the metadata
displayer = _metadata.MetadataDisplayer.with_model_file('textReadModel(handwritten).tflite')
print("Metadata populated:")
print(displayer.get_metadata_json())
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())
